In [1]:
import os
import torch

In [2]:
# read it in to inspect it
os.chdir('/mnt/data/projects/.immune/Personal/Transformers_Tutorial/')
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[:1000])  # print the first 1000 characters to check

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("All the unique characters:", ''.join(chars))
print("Vocab size:", vocab_size)

All the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size: 65


In [5]:
# creating a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: take a list of integers, output a string

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [6]:
# lets encode the entire text into the integer representation
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape,data.type)
print(data[:1000])  # first 1000 characters encoded as integers

torch.Size([1115394]) <built-in method type of Tensor object at 0x7f240642c680>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 

In [7]:
# splitting into train and val sets
n = int(0.9*len(data))  # first 90% will be
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8  # context length for predictions
train_data[:block_size+1] ### till 8 it will be train and 9th character is the target

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
print(decode(train_data[:block_size+1].tolist()))  # checking if decoding works

First Cit


In [10]:
x = train_data[:block_size]  # input
y = train_data[1:block_size+1]  # target
for t in range(block_size):
    context = x[:t+1]  # the context grows with t
    target = y[t]     # the target is always the next character
    print(f'when input is {context} the target: {target}')
    print(f"when input is {decode(context.tolist())!r} the target: {decode([target.tolist()])!r}")

when input is tensor([18]) the target: 47
when input is 'F' the target: 'i'
when input is tensor([18, 47]) the target: 56
when input is 'Fi' the target: 'r'
when input is tensor([18, 47, 56]) the target: 57
when input is 'Fir' the target: 's'
when input is tensor([18, 47, 56, 57]) the target: 58
when input is 'Firs' the target: 't'
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is 'First' the target: ' '
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is 'First ' the target: 'C'
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is 'First C' the target: 'i'
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
when input is 'First Ci' the target: 't'


In [ ]:
torch.manual_seed(1337)
batch_size = 4  # parallel sequences processing
block_size = 8  # context length for predictions

train_data.shape, val_data.shape
data = train_data
ix = torch.randint(len(data) - block_size, (batch_size,))
x = torch.stack([data[i:i+block_size] for i in ix])
y - torch.stack([data[i+1:i+block_size+1] for i in ix])
print("x:", x)
print("y:", y)

x: tensor([[43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39]])
y: tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [19]:
torch.manual_seed(1337)
batch_size = 4  # parallel sequences processing
block_size = 8  # context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # random starting indices
    x = torch.stack([data[i:i+block_size] for i in ix])  # input sequences
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])  # target sequences
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context} the target: {target}')
        print(f"when input is {decode(context.tolist())!r} the target: {decode([target.tolist()])!r}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is tensor([24]) the target: 43
when input is 'L' the target: 'e'
when input is tensor([24, 43]) the target: 58
when input is 'Le' the target: 't'
when input is tensor([24, 43, 58]) the target: 5
when input is 'Let' the target: "'"
when input is tensor([24, 43, 58,  5]) the target: 57
when input is "Let'" the target: 's'
when input is tensor([24, 43, 58,  5, 57]) the target: 1
when input is "Let's" the target: ' '
when input is tensor([24, 43, 58,  5, 57,  1]) the target: 46
when input is "Let's " the target: 'h'
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target: 43
when input i

In [60]:
# Constructing the bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # this is random initialization which will be learned during training.

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers where idx is xb and targets is yb
        logits = self.token_embedding_table(idx)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens): # Throw-away variable in loops so it does not fill the variable memory.
            # get the predictions
            logits, loss = self(idx) # Calls the Forward method
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1) #  draws a sample from the probability distribution. It returns the index of the next chosen token.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb) # PyTorch overloads the __call__() method of nn.Module. This calls the forward() method. 
print("logits:", logits)
print("loss:", loss)
print(logits.shape) # (B*T, C)
print(targets.shape) # (B*T, C)
# torch.zeroes is for new line character so we can initialize the generation from it.
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

logits: tensor([[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
        [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
        [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
        ...,
        [-2.1910, -0.7574,  1.9656,  ..., -0.3580,  0.8585, -0.6161],
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [-0.6787,  0.8662, -1.6433,  ...,  2.3671, -0.7775, -0.2586]],
       grad_fn=<ViewBackward0>)
loss: tensor(4.8786, grad_fn=<NllLossBackward0>)
torch.Size([32, 65])
torch.Size([32])

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [55]:
torch.zeros((1, 1))

tensor([[0.]])

In [53]:
logits[:,-1,:].shape

torch.Size([4, 65])

In [37]:
token_embedding=nn.Embedding(vocab_size, vocab_size)
token_embedding.weight

Parameter containing:
tensor([[-1.8001,  1.4810, -0.4281,  ...,  0.1231, -0.6002, -0.1704],
        [-0.3801,  0.7018, -0.2113,  ...,  2.6878, -0.0383, -0.5667],
        [ 1.3651, -0.5678,  0.7203,  ..., -0.1551,  0.9369, -1.4227],
        ...,
        [ 1.6218,  0.3064, -0.4684,  ...,  1.0780,  0.6130,  1.9363],
        [ 0.9706, -0.4319,  0.6869,  ..., -2.0820, -1.4026, -0.8381],
        [ 0.9202, -2.3964,  0.6574,  ..., -1.5965, -0.2411, -0.3512]],
       requires_grad=True)

In [43]:
logits = token_embedding(xb) # (B,T,C)

In [38]:
print(targets)
print(logits[0])
print(loss)
print(xb.shape) # B X T
print(yb.shape) # B X T

tensor([43, 58,  5, 57,  1, 46, 43, 39, 53, 56,  1, 58, 46, 39, 58,  1, 58,  1,
        58, 46, 39, 58,  1, 46, 17, 27, 10,  0, 21,  1, 54, 39])
tensor([-1.5101, -0.0948,  1.0927,  0.1505,  1.6347, -0.0518,  0.4996,  0.7216,
        -0.8968, -0.4122,  1.0030,  0.8508,  0.2178,  0.0328, -0.1699,  1.0659,
        -0.6177,  1.1824,  0.0214, -0.2154, -1.4623,  2.1707,  0.1624,  1.0296,
         0.4154,  0.6207,  0.2341, -0.0326,  1.0124,  1.5122, -0.3359,  0.2456,
         1.8682,  0.7536, -0.1177, -0.1967, -0.9552, -0.8995, -0.9583, -0.5945,
         0.1321, -0.5406,  0.1405, -0.7321,  1.1796,  1.3316, -0.2094,  0.0960,
         0.9040, -0.4032,  0.3027, -0.8034, -1.2537, -1.5195,  0.7446,  1.1914,
        -0.8061, -0.6290,  1.2447, -2.4400,  0.8408, -0.3993, -0.6126, -0.6597,
         0.7624], grad_fn=<SelectBackward0>)
tensor(4.8786, grad_fn=<NllLossBackward0>)
torch.Size([4, 8])
torch.Size([4, 8])


Training the model

In [62]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [67]:
batch_size = 32

for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    # optimize the model
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.428745746612549


In [69]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Morielo,
AUpur labeay anchr ran this; fin it QUKite bld suthe de: y eyomooigoun s st pponreiathathar y f s lld
SAnngomatthither. overcamireenurthissimu t I atin wnde pe
INu marelens; g, a atheis t olm'thyoodd ar tiansisernghanoueorif e. h'loo sothiese favereany amer d inour dnld CO,

LLUCHemavey.
y:
WI see nohen:
Whear'the byot gharce thes f.

NIINIOUSe haiorcope s y byes A:
Yomanomy t shayowshawhe lar trstithes iccong co nor aps CKithenino,
t sis pe'd he l sir be, cesokl that loripr igethan?
AT
